## Prep for running on colab

In [ ]:
!git clone https://github.com/ytsimon2004/deep_imgcls.git

In [ ]:
# dependencies installation
%cd deep_imgcls
!python setup_colab.py develop
!pip install -r requirements.txt

In [ ]:
# hack for setup.py run properly
!pip install ipykernel~=5.5.0

In [ ]:
from imgcls.classification.yolov8 import YoloUltralyticsPipeline, plot_image_seg, dir_ipy_imshow
from imgcls.io import *

from pathlib import Path

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Custom Pipeline for Running YOLOv8 Model for Image Classification

- Refer to the module documentation at `imgcls.classification.yolov8.pipeline.YoloUltralyticsPipeline`.

- For more information on training modes, visit [Ultralytics Documentation](https://docs.ultralytics.com/modes/train/).

- Avalible model type: `yolov8n`, `yolov8s`, `yolov8m`, `yolov8l`, `yolov8x`

- **Follow the data folder structure as outlined below**:
```
SOURCE_ROOT_DIR
│
├── dataset.yml (1)
├── run (2)
│   ├── predict*
│   │    └── test_set.csv (5)
│   ├── train*
│   │    └── weights (6)
│   └── *yolov8s.pt
│
├── test (3)
│   ├── img
│   ├── img_png
│   └── test_set.csv
│
└── train (4)
    ├── img
    ├── img_png
    ├── seg
    ├── seg_png
    └── train_set.csv
```
    (1) config yaml file for the custom path info/image labels
    
    (2) directory for model/train/evaluation output files
    
    (3) directory for test dataset
    
    (4) directory for train dataset

    (5) output results of classification
    
    (6) store the fine-tuned model weights

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# assume SOURCE_ROOT_DIR is under the /content/drive
yolo = YoloUltralyticsPipeline(root_dir=Path('/content/drive/MyDrive/comvis/imgcls'),
                               resize_dim=(500, 500),
                               model_type='yolov8n',
                               use_gpu=True,
                               epochs=100)

## **Step 1** - Clone batch raw .npy files to png in a separated folder

In [ ]:
yolo.clone_png_dir()

In [ ]:
# see example by indices
plot_image_seg(yolo.image_dir, index_range=(0, 5))

## **Step 2** - Generate the yaml for yolov8 config

In [ ]:
# <PATH> related values are user-specific
yolo.gen_yaml(verbose=True)

## **Step 3** - Detect the object edge from seg files and generate the yolo8 required label file for train dataset
- Each object example: \<class_id\> \<xc\> \<y_center\> \<width\> \<height\>

In [ ]:
# Use debug mode = True to see the examples
yolo.gen_label_txt(debug_mode=False)

## **Step 4/5** - Load a pretrained model for training & prediction

In [ ]:
# Option 1: If not yet fine-tuned the model.
yolo.yolo_train()
yolo.yolo_predict()

In [ ]:
# # Option 2: If a fine-tuned model already exists. directly load
model_path = ...  # e.g., */run/train/weights/best.pt
yolo.yolo_predict(model_path)

In [ ]:
# Brief interactive way to examine the predicted results using test dataset
predict_dir = yolo.image_dir.get_predict_dir(yolo.predict_filename)
dir_ipy_imshow(predict_dir, pattern='*.png')

## **Step 6** - Create csv for classified test dataset

In [ ]:
yolo.create_predicted_csv()